In [1]:
import numpy as np 
import pandas as pd 
import urllib 
import seaborn
import requests
import zipfile
from io import StringIO, BytesIO
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

#Set Pandas
pd.options.display.max_rows
pd.options.display.max_columns
pd.options.display.max_rows = 100
pd.options.display.max_columns = 60

In [2]:
df = pd.read_csv('/home/ggjinbuntu/lendingclubcsv/LoanStats3a.csv', skiprows=1, parse_dates=True, low_memory=False)
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,...,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/21/11 > to pay for prop...,other,personel,917xx,CA,20.00,0.0,Feb-1996,1.0,35.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,Current,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/21/11 > I plan on combi...,other,Personal,972xx,OR,17.94,0.0,Jan-1996,0.0,38.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [3]:
#check how many rows and columns dataframe have
df.shape

(42538, 111)

In [4]:
df=df.drop(['delinq_amnt', 'total_rec_late_fee', 'recoveries', 'out_prncp_inv', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'collection_recovery_fee', 'collections_12_mths_ex_med','last_credit_pull_d', 'last_pymnt_d','last_pymnt_amnt', 'initial_list_status','pymnt_plan','tax_liens', 'chargeoff_within_12_mths', 'application_type','policy_code', 'desc', 'addr_state', 'emp_title', 'title', 'zip_code', 'url', 'id', 'member_id', 'grade'], axis=1)

In [5]:
# drop columns that are already related with default
# drop columns where data consists of a value or similar situations

In [6]:
# 'out_prncp_inv', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 
# 'total_rec_late_fee', 'recoveries',
# 'collection_recovery_fee', 'collections_12_mths_ex_med','last_credit_pull_d', 'last_pymnt_d',
# 'last_pymnt_amnt', 'initial_list_status','pymnt_plan','tax_liens', 'chargeoff_within_12_mths'
# 'application_type','policy_code', 'desc', 'addr_state', 'emp_title', 'title', 'zip_code', 'url'
# 'id', 'member_id', 'grade', 'delinq_amnt'

In [7]:
# removing rows that does not have target value
# removing rows that has no data in such columns when the number of rows are to be removed is less than 30
df = df[pd.notnull(df['loan_status'])]
df = df[pd.notnull(df['total_acc'])]
df = df[pd.notnull(df['delinq_2yrs'])]

In [8]:
#check how many rows and columns dataframe have
df.shape

(42506, 83)

In [9]:
# Now we can see how many NaN values on each column
# There are many columns with 0 information
df.count()

loan_amnt                         42506
funded_amnt                       42506
funded_amnt_inv                   42506
term                              42506
int_rate                          42506
installment                       42506
sub_grade                         42506
emp_length                        42506
home_ownership                    42506
annual_inc                        42506
verification_status               42506
issue_d                           42506
loan_status                       42506
purpose                           42506
dti                               42506
delinq_2yrs                       42506
earliest_cr_line                  42506
inq_last_6mths                    42506
mths_since_last_delinq            15609
mths_since_last_record             3651
open_acc                          42506
pub_rec                           42506
revol_bal                         42506
revol_util                        42445
total_acc                         42506


In [10]:
# drop columns if datasets include NaN more than 40% set's elements
df = df.dropna(thresh=len(df)*0.40, axis=1)

In [11]:
#check how many rows and columns dataframe have
df.shape

(42506, 26)

In [12]:
# Now we can see how many non-NaN values each column has
df.count()

loan_amnt               42506
funded_amnt             42506
funded_amnt_inv         42506
term                    42506
int_rate                42506
installment             42506
sub_grade               42506
emp_length              42506
home_ownership          42506
annual_inc              42506
verification_status     42506
issue_d                 42506
loan_status             42506
purpose                 42506
dti                     42506
delinq_2yrs             42506
earliest_cr_line        42506
inq_last_6mths          42506
open_acc                42506
pub_rec                 42506
revol_bal               42506
revol_util              42445
total_acc               42506
out_prncp               42506
acc_now_delinq          42506
pub_rec_bankruptcies    41170
dtype: int64

In [13]:
df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,acc_now_delinq,pub_rec_bankruptcies
0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B2,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,credit_card,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,0.00,0.0,0.0
1,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C4,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,car,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,0.00,0.0,0.0
2,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C5,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,small_business,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,0.00,0.0,0.0
3,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C1,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,other,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,0.00,0.0,0.0
4,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B5,1 year,RENT,80000.0,Source Verified,Dec-2011,Current,other,17.94,0.0,Jan-1996,0.0,15.0,0.0,27783.0,53.9%,38.0,398.75,0.0,0.0


In [14]:
# revol_util and pub_rec_bankruptcies have 61, 1336 NaN values for each
df.isnull().sum()

loan_amnt                  0
funded_amnt                0
funded_amnt_inv            0
term                       0
int_rate                   0
installment                0
sub_grade                  0
emp_length                 0
home_ownership             0
annual_inc                 0
verification_status        0
issue_d                    0
loan_status                0
purpose                    0
dti                        0
delinq_2yrs                0
earliest_cr_line           0
inq_last_6mths             0
open_acc                   0
pub_rec                    0
revol_bal                  0
revol_util                61
total_acc                  0
out_prncp                  0
acc_now_delinq             0
pub_rec_bankruptcies    1336
dtype: int64

In [15]:
# See how many NaN exist on the column
df['pub_rec_bankruptcies'].value_counts(dropna=False)

 0.0    39316
 1.0     1846
NaN      1336
 2.0        8
Name: pub_rec_bankruptcies, dtype: int64

In [16]:
df['revol_util'].value_counts(dropna=False)

0%        1070
40.7%       65
0.2%        64
63%         63
66.6%       62
0.1%        61
70.4%       61
NaN         61
37.6%       60
64.6%       60
35.3%       59
66.7%       59
31.2%       59
46.4%       59
76.6%       58
25.5%       58
46.6%       58
55.8%       58
57.4%       58
89.5%       58
61%         58
78.7%       58
68.6%       57
66.9%       57
49.8%       57
65.9%       57
72.1%       57
48.9%       57
34.4%       57
57.7%       56
55.5%       56
51.6%       56
48%         56
54.6%       56
45.6%       56
64.8%       56
70.1%       56
47.8%       56
55.2%       56
41%         56
30.9%       56
54.3%       56
54.1%       56
57.5%       55
43.4%       55
70.8%       55
48.2%       55
74.9%       55
89.9%       55
51.1%       55
          ... 
94.46%       1
32.04%       1
11.63%       1
36.94%       1
8.58%        1
0.54%        1
30.19%       1
7.64%        1
5.79%        1
36.78%       1
103.5%       1
49.63%       1
21.72%       1
105.3%       1
46.74%       1
0.46%     

In [17]:
# fill NaN with 0.0 since 0.0 is the mode of the dataset and it will not affect the whole data much
df['pub_rec_bankruptcies'].fillna(df['pub_rec_bankruptcies'].mode()[0], inplace=True)
df['revol_util'].fillna(df['revol_util'].mode()[0], inplace=True)

In [18]:
# check whether NaNs are gone
df['pub_rec_bankruptcies'].unique()

array([ 0.,  1.,  2.])

In [19]:
# remove non-numeric values and change data types to float type
df['revol_util'] = df['revol_util'].str[:-1].astype(float)
df['int_rate'] = df['int_rate'].str[:-1].astype(float)
df['term'] = df['term'].str[:-6].astype(float)
df['emp_length'] = df['emp_length'].str.extract('(\d+)')
# emp_length has literally 'Nan' Value, I replace 'Nan' with 0 since I suspect Nan means they do not have a job based on DataDictionary file
df['emp_length'] = df['emp_length'].fillna(0, inplace=False)
df['emp_length'] = df['emp_length'].astype(float)



/home/ggjinbuntu/anaconda3/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:5: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


In [20]:
# imp_freq = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)

In [21]:
# len(imp_freq.fit_transform(np.array(df['revol_util']))[0])

In [22]:
# see if any colums havea  null value
df.isnull().sum()

loan_amnt               0
funded_amnt             0
funded_amnt_inv         0
term                    0
int_rate                0
installment             0
sub_grade               0
emp_length              0
home_ownership          0
annual_inc              0
verification_status     0
issue_d                 0
loan_status             0
purpose                 0
dti                     0
delinq_2yrs             0
earliest_cr_line        0
inq_last_6mths          0
open_acc                0
pub_rec                 0
revol_bal               0
revol_util              0
total_acc               0
out_prncp               0
acc_now_delinq          0
pub_rec_bankruptcies    0
dtype: int64

In [23]:
df.dtypes

loan_amnt               float64
funded_amnt             float64
funded_amnt_inv         float64
term                    float64
int_rate                float64
installment             float64
sub_grade                object
emp_length              float64
home_ownership           object
annual_inc              float64
verification_status      object
issue_d                  object
loan_status              object
purpose                  object
dti                     float64
delinq_2yrs             float64
earliest_cr_line         object
inq_last_6mths          float64
open_acc                float64
pub_rec                 float64
revol_bal               float64
revol_util              float64
total_acc               float64
out_prncp               float64
acc_now_delinq          float64
pub_rec_bankruptcies    float64
dtype: object

In [24]:
# change them into ratio
df['funded_amnt_inv'] = df['funded_amnt_inv']/df['funded_amnt']

In [25]:
# change subgrade into value since it has ordinal feature
def conv_dict1(x):
    res = 5 - int(x[1:]) # since big number is better
    converter = {'G':0, 'F':1, 'E':2, 'D':3, 'C':4, 'B':5, 'A':6}
    res += 6*converter[x[0]]  # base6
    return res

df['sub_grade'] = df['sub_grade'].apply(conv_dict1)

In [26]:
df.shape

(42506, 26)

In [27]:
df['home_ownership'].value_counts(dropna=False)

RENT        20164
MORTGAGE    18952
OWN          3250
OTHER         136
NONE            4
Name: home_ownership, dtype: int64

In [28]:
# make it ready to apply one-hot-encoding
def conv_dict_home_ownership(x):
    converter = {'RENT':0, 'OWN':1, 'MORTGAGE':2, 'OTHER':3, 'NONE':4}
    res = converter[x]
    return res

In [29]:
df['home_ownership'] = df['home_ownership'].apply(conv_dict_home_ownership)

In [30]:
df['home_ownership'].value_counts(dropna=False)

0    20164
2    18952
1     3250
3      136
4        4
Name: home_ownership, dtype: int64

In [31]:
ohe_home_ownership = OneHotEncoder()

In [32]:
df['home_ownership'].values[:, np.newaxis]

array([[0],
       [0],
       [0],
       ..., 
       [2],
       [0],
       [2]])

In [33]:
ohe_home_ownership.fit(df['home_ownership'].values[:, np.newaxis])

OneHotEncoder(categorical_features='all', dtype=<class 'float'>,
       handle_unknown='error', n_values='auto', sparse=True)

In [34]:
ohe_home_ownership.transform(df['home_ownership'].values[:, np.newaxis]).toarray()

array([[ 1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  1.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.]])

In [35]:
home_ownership_onehot = ohe_home_ownership.transform(df['home_ownership'].values[:, np.newaxis]).toarray()

In [36]:
home_ownership_onehot

array([[ 1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  1.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.]])

In [37]:
home_owner_df = pd.DataFrame(home_ownership_onehot, columns=['RENT', 'OWN', 'MORTGAGE', 'OTHER', 'NONE'])
home_owner_df.tail()

,RENT,OWN,MORTGAGE,OTHER,NONE
42501,0.0,1.0,0.0,0.0,0.0
42502,1.0,0.0,0.0,0.0,0.0
42503,0.0,0.0,1.0,0.0,0.0
42504,1.0,0.0,0.0,0.0,0.0
42505,0.0,0.0,1.0,0.0,0.0


In [38]:
# reset index before perform concatenate
df = df.reset_index(drop=True)

In [39]:
df = pd.concat([df, home_owner_df], axis=1)

In [40]:
df['MORTGAGE'].value_counts(dropna=False)

0.0    23554
1.0    18952
Name: MORTGAGE, dtype: int64

In [41]:
# now drop home_ownership column
df = df.drop(['home_ownership'], axis=1)

In [42]:
df['verification_status'].unique()

array(['Verified', 'Source Verified', 'Not Verified'], dtype=object)

In [43]:
# Give each verification_status numeric value
def conv_dict_verification_status(x):
    converter = {'Not Verified':0, 'Source Verified':1, 'Verified':2}
    res = int(converter[x])
    return res

In [44]:
df['verification_status'] = df['verification_status'].apply(conv_dict_verification_status)

In [45]:
df['issue_month'] = df['issue_d'].str[:3].astype(str)

In [46]:
df['issue_month'].unique()

array(['Dec', 'Nov', 'Oct', 'Sep', 'Aug', 'Jul', 'Jun', 'May', 'Apr',
       'Mar', 'Feb', 'Jan'], dtype=object)

In [47]:
# drop issue_d colum after extract month
df = df.drop(['issue_d'], axis=1)

In [48]:
# Each month will have their own class
def conv_dict_verification_status(x):
    converter = {'Jan':0, 'Feb':1, 'Mar':2, 'Apr':3, 'May':4, 'Jun':5, 'Jul':6, 'Aug':7, 'Sep':8, 'Oct':9, 'Nov':10, 'Dec':11}
    res = int(converter[x])
    return res

In [49]:
df['issue_month'] = df['issue_month'].apply(conv_dict_verification_status)

In [50]:
issue_month_ohe = OneHotEncoder()

In [51]:
issue_month_ohe.fit_transform(df['issue_month'].values[:,np.newaxis]).toarray()

array([[ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [52]:
_ = pd.DataFrame(issue_month_ohe.fit_transform(df['issue_month'].values[:,np.newaxis]).toarray(), columns=["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"])

In [53]:
_

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [54]:
df = pd.concat([df, _], axis=1)

In [55]:
df.tail()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,sub_grade,emp_length,annual_inc,verification_status,loan_status,purpose,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,acc_now_delinq,pub_rec_bankruptcies,RENT,OWN,MORTGAGE,OTHER,NONE,issue_month,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
42501,5350.0,5350.0,0.116822,36.0,13.12,180.57,18,1.0,4000.0,0,Does not meet the credit policy. Status:Fully ...,car,15.00,0.0,Dec-2006,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,6,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
42502,10000.0,10000.0,0.035000,36.0,14.70,345.18,12,1.0,50000.0,0,Does not meet the credit policy. Status:Fully ...,educational,7.22,2.0,Sep-1999,0.0,14.0,0.0,10025.0,85.0,20.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,6,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
42503,2000.0,2000.0,0.637500,36.0,7.12,61.87,40,7.0,150000.0,0,Does not meet the credit policy. Status:Fully ...,debt_consolidation,5.60,0.0,Mar-1984,0.0,7.0,0.0,150786.0,2.2,16.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
42504,6000.0,6000.0,0.108333,36.0,10.59,195.28,27,1.0,20000.0,0,Does not meet the credit policy. Status:Fully ...,debt_consolidation,12.90,0.0,Jan-1996,4.0,5.0,0.0,13660.0,66.0,6.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
42505,4400.0,4400.0,0.318182,36.0,9.64,141.25,31,2.0,30000.0,0,Does not meet the credit policy. Status:Fully ...,debt_consolidation,3.72,0.0,Jul-2004,0.0,4.0,0.0,3493.0,63.5,5.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
df['loan_status'].value_counts()

Fully Paid                                             33276
Charged Off                                             5640
Does not meet the credit policy. Status:Fully Paid      1962
Current                                                  827
Does not meet the credit policy. Status:Charged Off      758
In Grace Period                                           18
Late (31-120 days)                                        15
Late (16-30 days)                                          5
Default                                                    5
Name: loan_status, dtype: int64

In [57]:
df['loan_status'].unique()

array(['Fully Paid', 'Charged Off', 'Current', 'In Grace Period',
       'Late (31-120 days)', 'Late (16-30 days)', 'Default',
       'Does not meet the credit policy. Status:Fully Paid',
       'Does not meet the credit policy. Status:Charged Off'], dtype=object)

In [58]:
# Make a target value
def conv_dict_loan_status(x):
    converter = {'Fully Paid':0 , 'Charged Off': 1, 'Current':0, 'In Grace Period':1, 
                 'Late (31-120 days)':1 , 'Late (16-30 days)':1, 'Default':1,
                 'Does not meet the credit policy. Status:Fully Paid':0,
       'Does not meet the credit policy. Status:Charged Off':1}
    res = int(converter[x])
    return res

In [59]:
df['loan_status'] = df['loan_status'].apply(conv_dict_loan_status)

In [60]:
df['loan_status'].unique()

array([0, 1])

In [61]:
df.columns

Index(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'sub_grade', 'emp_length', 'annual_inc',
       'verification_status', 'loan_status', 'purpose', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'inq_last_6mths', 'open_acc', 'pub_rec',
       'revol_bal', 'revol_util', 'total_acc', 'out_prncp', 'acc_now_delinq',
       'pub_rec_bankruptcies', 'RENT', 'OWN', 'MORTGAGE', 'OTHER', 'NONE',
       'issue_month', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug',
       'Sep', 'Oct', 'Nov', 'Dec'],
      dtype='object')

In [62]:
# classify purpose column with LabelEncoder

In [63]:
df['purpose'].unique()

array(['credit_card', 'car', 'small_business', 'other', 'wedding',
       'debt_consolidation', 'home_improvement', 'major_purchase',
       'medical', 'moving', 'vacation', 'house', 'renewable_energy',
       'educational'], dtype=object)

In [64]:
le = LabelEncoder()

In [65]:
le.fit_transform(df['purpose'].values)

array([ 1,  0, 11, ...,  2,  2,  2])

In [66]:
le.classes_

array(['car', 'credit_card', 'debt_consolidation', 'educational',
       'home_improvement', 'house', 'major_purchase', 'medical', 'moving',
       'other', 'renewable_energy', 'small_business', 'vacation', 'wedding'], dtype=object)

In [67]:
le.fit_transform(df['purpose'].values)[:,np.newaxis]

array([[ 1],
       [ 0],
       [11],
       ..., 
       [ 2],
       [ 2],
       [ 2]])

In [68]:
# apply onehotencoder

In [69]:
ohe = OneHotEncoder()
ohe.fit_transform(le.fit_transform(df['purpose'].values)[:,np.newaxis]).toarray()

array([[ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       ..., 
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.]])

In [70]:
_ = pd.DataFrame(ohe.fit_transform(le.fit_transform(df['purpose'].values)[:,np.newaxis]).toarray(), 
                 columns=le.classes_)

In [71]:
_

,car,credit_card,debt_consolidation,educational,home_improvement,house,major_purchase,medical,moving,other,renewable_energy,small_business,vacation,wedding
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [72]:
df = pd.concat([_, df], axis=1)

In [73]:
# drop purpose column after classifying purpose
df = df.drop("purpose", axis=1)

In [74]:
df.shape

(42506, 55)

In [75]:
df['dti'].dtype

dtype('float64')

In [76]:
df['delinq_2yrs'].unique()

array([  0.,   2.,   3.,   1.,   4.,   6.,   5.,   8.,   7.,   9.,  11.,
        13.])

In [77]:
df['earliest_cr_line'] = df['earliest_cr_line'].str[4:].astype(int)

In [78]:
df['earliest_cr_line'] = 2016 - df['earliest_cr_line']

In [79]:
df['pub_rec_bankruptcies'].value_counts()

0.0    40652
1.0     1846
2.0        8
Name: pub_rec_bankruptcies, dtype: int64

In [80]:
# see if this column has Null value or not, if the sume is 0, it means it does not have any Null value
df['pub_rec_bankruptcies'].isnull().sum()

0

In [81]:
df.columns

Index(['car', 'credit_card', 'debt_consolidation', 'educational',
       'home_improvement', 'house', 'major_purchase', 'medical', 'moving',
       'other', 'renewable_energy', 'small_business', 'vacation', 'wedding',
       'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'sub_grade', 'emp_length', 'annual_inc',
       'verification_status', 'loan_status', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'inq_last_6mths', 'open_acc', 'pub_rec',
       'revol_bal', 'revol_util', 'total_acc', 'out_prncp', 'acc_now_delinq',
       'pub_rec_bankruptcies', 'RENT', 'OWN', 'MORTGAGE', 'OTHER', 'NONE',
       'issue_month', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug',
       'Sep', 'Oct', 'Nov', 'Dec'],
      dtype='object')

In [83]:
df

,car,credit_card,debt_consolidation,educational,home_improvement,house,major_purchase,medical,moving,other,renewable_energy,small_business,vacation,wedding,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,sub_grade,emp_length,annual_inc,verification_status,loan_status,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,acc_now_delinq,pub_rec_bankruptcies,RENT,OWN,MORTGAGE,OTHER,NONE,issue_month,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5000.0,5000.0,0.995000,36.0,10.65,162.87,33,10.0,24000.00,2,0,27.65,0.0,31,1.0,3.0,0.0,13648.0,83.70,9.0,0.00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2500.0,2500.0,1.000000,60.0,15.27,59.83,25,1.0,30000.00,1,1,1.00,0.0,17,5.0,3.0,0.0,1687.0,9.40,4.0,0.00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2400.0,2400.0,1.000000,36.0,15.96,84.33,24,10.0,12252.00,0,0,8.72,0.0,15,2.0,2.0,0.0,2956.0,98.50,10.0,0.00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,10000.0,10000.0,1.000000,36.0,13.49,339.31,28,10.0,49200.00,1,0,20.00,0.0,20,1.0,10.0,0.0,5598.0,21.00,37.0,0.00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3000.0,3000.0,1.000000,60.0,12.69,67.79,30,1.0,80000.00,1,0,17.94,0.0,20,0.0,15.0,0.0,27783.0,53.90,38.0,398.75,0.0,0.0,1.0,0.0,0.0,0.0,0.0,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5000.0,5000.0,1.000000,36.0,7.90,156.46,37,3.0,36000.00,1,0,11.20,0.0,12,3.0,9.0,0.0,7963.0,28.30,12.0,0.00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7000.0,7000.0,1.000000,60.0,15.96,170.08,24,8.0,47004.00,0,0,23.51,0.0,11,1.0,7.0,0.0,17726.0,85.60,11.0,0.00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3000.0,3000.0,1.000000,36.0,18.64,109.43,16,9.0,48000.00,1,0,5.35,0.0,9,2.0,4.0,0.0,8221.0,87.50,4.0,0.00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,5600.0,5600.0,1.000000,60.0,21.28,152.39,9,4.0,40000.00,1,1,5.55,0.0,12,2.0,11.0,0.0,5210.0,32.60,13.0,0.00,0.0,0.0,0.0,1.0,0.0,0.0,0.0,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,5375.0,5375.0,0.995349,60.0,12.69,121.45,30,1.0,15000.00,2,1,18.08,0.0,12,0.0,2.0,0.0,9279.0,36.50,3.0,0.00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [84]:
df.to_csv('/home/ggjinbuntu/lendingclubcsv/preproccssedlendq1.csv', index=False)